In [3]:
import numpy as np
import pandas as pd
import glob
import os

In [13]:
path = r'../raw_data/dilma/' # use your path
all_files = glob.glob(os.path.join(path, "*.csv"))     # advisable to use os.path.join as this makes concatenation OS independent

df_from_each_file = (pd.read_csv(f) for f in all_files)
df   = pd.concat(df_from_each_file, ignore_index=True)

In [5]:
df.shape

(427955, 25)

In [6]:
def clean_tweet(tweet):

    tweet = tweet.lower()
    tweet = re.sub(r"\d+", "", tweet)
    tweet = re.sub(r'\@[a-zA-Z0-9]*', ' ', tweet) # remove username start with @
    tweet = re.sub(r'https?:\/\/[a-zA-Z0-9@:%._\/+~#=?&;-]*', ' ', tweet) # remove link in the tweet
    tweet = re.sub(r'\$[a-zA-Z0-9]*', ' ', tweet) # remove the variable start with $
    for punctuation in new_punc: # remove panctuations 
            tweet = tweet.replace(punctuation, '')
    tweet = ' '.join( [w for w in tweet.split() if len(w)>1] ) #remove one letter words
    tweet = ''.join(c[0] for c in itertools.groupby(tweet)) #remove duplicated letters
    tweet = unidecode.unidecode(tweet)  # normalizar as letras com acentos 
    #tweet = ' '.join(word for word in tweet.split(' ') if word not in stop_words) # remove stopwords

    return tweet

In [7]:
import nltk
from nltk.corpus import stopwords
import re
import string
import itertools
import unidecode

In [8]:
%%time
new_punc = list(string.punctuation)
del new_punc[2]
stop_words = stopwords.words('portuguese')
stop_words.remove('não')
addicional = [
            'd', 'ta', 'q', 'tah', 'tao', 'eh', 'vc', 'voce',
            'pq', 'quede', 'mto', 'mt', 'bj', 'bjs','vcs','bb','pra','ai',
            'b', 'sao', 'axo', 'mano', 'ae', 'neh', 'ai','la','ja','so',
            'ne', 'no', 'iai', 'tbm', 'msm', 'jah', 'yahoo', 'yahoobr','rt']
stop_words.extend(addicional)
df['content'] = df['content'].astype(str)
df["content_clean"]=df["content"].apply(lambda x:clean_tweet(x))
df = df.drop_duplicates(subset="content_clean",keep="first")
df=df[df["lang"]=="pt"]
df=df[["date","username","content_clean"]]
df['date'] = pd.to_datetime(df['date'])
df

CPU times: user 18.5 s, sys: 179 ms, total: 18.7 s
Wall time: 18.8 s


,date,username,content_clean
0,2014-05-11 13:42:19,dilmabr,uma data para marcar reconhecimento carinho re...
1,2014-05-11 13:41:51,dilmabr,como filha mae avo sei como cada uma de voces ...
2,2014-05-11 13:41:37,dilmabr,hoje envio um abraco cheio de alegria todas as...
3,2014-08-09 20:08:07,dilmabr,as imagens da agenda com hj em osasco
4,2014-08-09 13:20:06,dilmabr,para quem nao esta em osasco caminhada sera tr...
...,...,...,...
427948,2014-09-10 00:14:50,JCRuckill,roubar dinheiro de pobre feio viu
427949,2014-09-10 00:14:04,JCRuckill,para colocar laque no capacete com dinheiro do...
427950,2014-09-10 00:13:17,JCRuckill,eta esquerda de meia tigela que gosta de um di...
427951,2014-09-10 00:11:53,LFDivulga,nao iria melhorar internet do brasil ah pois n...


In [9]:
from tqdm import tqdm_notebook
from sentence_transformers import SentenceTransformer
from bertopic import BERTopic
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer

2022-06-06 13:39:04.171034: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-06 13:39:04.171054: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [10]:
%%time
neighbour_cluster = 15
sentence_model = SentenceTransformer('paraphrase-MiniLM-L6-v2') # paraphrase-MiniLM-L6-v2
hdbscan_model = HDBSCAN(min_cluster_size=neighbour_cluster, metric='euclidean', #min_cluster_size=10,100
                        cluster_selection_method='eom', prediction_data=True)
umap_model = UMAP(n_neighbors=neighbour_cluster, n_components=30, min_dist=0.0, metric='cosine', low_memory=True, random_state=42) # n_neighbors=15,30, 100
vectorizer_model = CountVectorizer(ngram_range=(1, 2), stop_words=stop_words, min_df=5)         # min_df means "ignore terms that appear in less than n documents"
topic_model = BERTopic(#language="multilingual",
                       top_n_words=10,                                                    
                       min_topic_size=11,
                       nr_topics=11,                                                           # Don't set nr_topics if you're already converging topics to pre-determined labels
                       low_memory=True,                                                            # If you have a big dataset set low_memory to True - but it's slower
                       calculate_probabilities=False,  # Set to False if big data
                       seed_topic_list=None,                                                       # If you want to converge to a list of your own topics
                       embedding_model=sentence_model,
                       umap_model=umap_model,
                       hdbscan_model=hdbscan_model,
                       vectorizer_model=vectorizer_model,
                       verbose=True                                                                # Get logging
                        
                      )

CPU times: user 414 ms, sys: 130 ms, total: 544 ms
Wall time: 8.75 s


In [11]:
docs = df['content_clean'].to_list()  
topics, probs = topic_model.fit_transform(docs) 

Batches:   0%|          | 0/9455 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
freq = topic_model.get_topic_info() 
freq.head(10)

In [ ]:
n = 1
display(topic_model.get_topic(n))
#display(topic_model.get_representative_docs(n))

In [ ]:
fig_topics = topic_model.visualize_topics()
fig_topics.write_html("dilma_TopicDistance.html")

In [ ]:
fig_hierarchy = topic_model.visualize_hierarchy()
fig_hierarchy.write_html("dilma_hierarchicalCluster.html")

In [ ]:
topics_over_time = topic_model.topics_over_time(df['content_clean'].tolist(), topics, df['date'].tolist(), nr_bins=20)
topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=12)
fig_overtime = topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=12)
fig_overtime.write_html("dilma_topic_overtime.html")

In [ ]:
fig_heatmap = topic_model.visualize_heatmap()
fig_heatmap.write_html("dilma_topic_heatmap.html")

In [ ]:
fig_wordScores = topic_model.visualize_barchart()
fig_wordScores.write_html("dilma_topic_wordScores.html")

In [ ]:
fig_termRank = topic_model.visualize_term_rank()
fig_wordScores.write_html("dilma_topic_wordScores.html")